# FMD Scorecard KPIs
This notebook has a few objectives. 

- Come as close as we can to replicating the results reported in previous years. This lets us know that we are all looking at the same data and seeing the same thing
- Show the results we'll get with modified results 

Note that in the past, the Department of General Services has used a multi-phase process to compute KPIs, including SQL Queries and Excel formulas. No one person could see the entire process, beginning with the data in the database (the input) and ending with reported KPI. 

Part of the intention of this notebook is to build _transparency_ and _reproducibility_ by capturing the entire end-to-end process in one place.

## Setup

### Import packages

In [1]:
# workhorse modules
import pandas as pd
import numpy as np
from datetime import timedelta, datetime
import re
from pathlib import Path

# import datadotworld as dw
import pyodbc  # for accessing the database directly
import seaborn as sns

# local utility functions
from utils import *
from private.config import config

### Set pandas options
This makes Pandas print all rows and columns to the output when requested.

In [2]:
set_pd_params()

pd.options.mode.chained_assignment = None  # default='warn'

### Import the data from Archibus database
Data is a copy of Archibus's `wrhwr` table. To see the exact query used to generate the input data, see `/sql/input_for_FMD_KPIs.sql`.

In [3]:
query_path = Path.cwd() / "sql" / "input_for_FMD_KPIs.sql"
kpis_raw = get_kpi_data(config, query_path)

print(f"The KPIs raw dataframe has {kpis_raw.shape[0]:,} rows.")
kpis_raw.sample(3, random_state=444)

The KPIs raw dataframe has 104,495 rows.


,wr_id,date_requested,time_requested,date_completed,time_completed,date_closed,pmp_id,bl_id,cost_total,cost_labor,cost_parts,problem_type,requestor,supervisor,po_number,invoice_number,release_number,name,pmp_id,status
92667,125800,2019-09-13,1899-12-30 10:24:21,2019-09-16,1899-12-30 05:45:51,2019-10-08 07:22:40.450,None,B00163,60.0,60.0,0.0,PLUMB/OTHER,JIMMY.HOLTHAUS,JOHN.HAMMOCK,None,None,None,The Weinberg Housing and Resource Center,None,Clo
77987,108006,2018-07-11,1899-12-30 16:17:44,2018-08-06,1899-12-30 13:33:39,2018-10-01 09:57:37.600,None,B00057,120.0,120.0,0.0,OTHER,LORETTA.BROWN,JIMMY.HOLTHAUS,None,None,None,Abel Wolman Muncipal Building,None,Clo
62630,91707,2017-08-01,1899-12-30 13:26:56,NaT,NaT,NaT,None,B00030,0.0,0.0,0.0,HVAC,CYNTHIA.GORDON,None,None,None,None,Baltimore City Police South Western District,None,Can


## Data cleaning

### Basic cleaning
- removes white spaces in strings to facilitate matching, 
- drops rows with no problem type, 
- renames a few columns

In [4]:
# apply the tidy up function
wr_tidy = tidy_up_wr(kpis_raw)

print(f"The tidied work orders dataframe has {wr_tidy.shape[0]:,} rows.")

The tidied work orders dataframe has 104,105 rows.


### Remove duplicate work orders
Removes rows where technician says WR is a duplicate in the description, and the status is "Canceled", "Closed", or "Rejected". Let's hold off on this step while replicating the traditional KPI.

In [5]:
# This copies the previous step instead of applying deduplication.
wr_deduped = wr_tidy.copy()

# wr_deduped = drop_dupes(wr_tidy)
# print(f"The deduped work orders dataframe has {wr_deduped.shape[0]:,} rows.")
# print(
#     f"Removing duplicates has cut {wr_tidy.shape[0] - wr_deduped.shape[0]:,} rows from the work orders dataframe."
# )

### Combine date and time columns to get timestamps
This takes the date from a date column and the time from a time column and combines them into a single timestamp.

This transformation allows us to know the time to completion with greater precision. 

In [6]:
# glue the date and time for request
wr_dt = glue_date_time(wr_deduped, "date_requested", "time_requested", "requested_dt")

# glue the date and time for completion
wr_dt = glue_date_time(wr_dt, "date_completed", "time_completed", "completed_dt")

# convert "date closed_order" to date time (this column has no time information)
wr_dt["date_closed"] = wr_dt["date_closed"].astype("datetime64")

### Examine the cleaned data

In [7]:
wr_dt[
    ["wr_id", "problem_type", "requested_dt", "completed_dt", "date_closed", "status",]
].sample(3, random_state=451)

,wr_id,problem_type,requested_dt,completed_dt,date_closed,status
63082,92156,CARPENTRY,2017-08-10 12:15:29,2017-08-29 14:29:27,2017-10-02 11:40:12.807,Clo
42753,67440,OTHER,2016-06-10 14:13:18,2016-06-10 14:14:07,2016-06-30 10:58:33.043,Clo
94171,127327,PREVENTIVE MAINT,2019-10-23 07:00:10,2019-10-24 09:49:01,NaT,Com


## Data preparation

### Include days to completion

In [11]:
wr_durations = compute_days_to_completion(wr_dt)

In [12]:
wr_durations[
    [
        "wr_id",
        "problem_type",
        "requested_dt",
        "completed_dt",
        "date_closed",
        "days_to_completion",
        "status",
    ]
].sample(3, random_state=446)

,wr_id,problem_type,requested_dt,completed_dt,date_closed,days_to_completion,status
requested_dt,,,,,,,
2016-10-05 11:24:17,75820,DOOR,2016-10-05 11:24:17,2016-10-21 05:37:41,2016-10-21 06:29:18.977,15.76,Clo
2017-01-25 16:32:56,82811,PAINT,2017-01-25 16:32:56,2017-01-31 07:29:11,2017-02-02 05:54:49.913,5.62,Clo
2018-03-07 10:28:06,102240,ELEC/GENERAL,2018-03-07 10:28:06,2018-03-07 10:29:09,2018-03-14 08:30:18.777,0.00,Clo


### Decision point: fiscal year
Note that the function `entirely_within_fiscal_year()` keeps only those rows where the work order was requested and closed in the same fiscal year. __Other rows that straddle two fiscal years are dropped__.

For comparison, I've included the function `add_fiscal_year()`, which derives the fiscal year from the request date or from the completion date — and drops no rows.

In [13]:
# this is the traditional way
wr_fy, dropped_rows = entirely_within_fiscal_year(wr_durations)

# this is a new way
# wr_fy = add_fiscal_year(wr_durations)

In [25]:
print(wr_fy[wr_fy['requested_dt'] >= '07-01-2018']['days_to_completion'].mean())
print(dropped_rows[dropped_rows['requested_dt'] >= '07-01-2018']['days_to_completion'].mean())

30.072454381617483
254.0545239520958


In [ ]:
dropped_row_count = len(wr_durations) - len(wr_fy)
percent_rows_dropped = round(dropped_row_count / len(wr_durations) * 100, 2)

print(
    f"Limiting analysis to work orders entirely within one FY drops {dropped_row_count:,} rows from the data."
)
print(f"The dropped rows account for {percent_rows_dropped:,}% of the data.")
print(f"The prepared dataset contains {len(wr_fy):,} rows.")

In [ ]:
wr_fy[
    [
        "wr_id",
        "problem_type",
        "days_to_completion",
        "requested_dt",
        "completed_dt",
        "fiscal_year",
    ]
].sample(3, random_state=444)

## KPI 1: % PMs completed on time 
The goal here is to filter the data down to preventive maintenance only, and then show how many are completed before a given benchmark.

### Filter to PM only, and for relevant fiscal years only

In [ ]:
PM_list = [
    "HVAC|PM",
    "BUILDING|PM",
    "PREVENTIVE MAINT",
    "FUEL INSPECTION",
    "BUILDING INTERIOR INSPECTION",
    "INSPECTION",
    "FUEL INSPECTION",
]

cond_fy = wr_fy["fiscal_year"].isin(range(2016, 2021))
cond_pm = wr_fy["problem_type"].isin(PM_list)

wr_filtered = wr_fy[cond_fy & cond_pm]

print(f"The filtered work orders dataframe has {wr_filtered.shape[0]:,} rows.")

#### Compute the benchmark and add 'is_on_time' column

In [ ]:
def compute_is_on_time(df, benchmark):
    df = df.copy()
    df["is_on_time"] = df["days_to_completion"] <= benchmark
    return df


wr_on_time = compute_is_on_time(wr_filtered, 26)

#### Group by fiscal year and get % on time

In [ ]:
def compute_kpi_table(df, label_for_totals):
    df = df.copy()
    table_df = df.groupby("fiscal_year")[["is_on_time"]].agg(["mean", "count"])
    table_df.columns = table_df.columns.droplevel(0)
    # table_df["mean"] = pm_compliance["mean"].round(2)
    table_df["mean"] = table_df["mean"].apply(lambda x: round(x * 100, 2))
    table_df = table_df.rename(
        columns={"mean": "percent_on_time", "count": label_for_totals}
    )
    return table_df


pm_compliance = compute_kpi_table(wr_on_time, "total_PMs")
pm_compliance

## KPI 2: PM:CM ratio

The two lists below contain the exact same problem types mentioned in last year's scorecard. So we would expect to be able to replicate last year's results closely.

In [ ]:
CM_list = [
    "BOILER",
    "CHILLERS",
    "COOLING TOWERS",
    "HVAC",
    "HVAC INFRASTRUCTURE",
    "HVAC|REPAIR",
]

PM_list = [
    "HVAC|PM",
    "PREVENTIVE MAINT",
    # "BUILDING|PM",
    # "FUEL INSPECTION",
    # "BUILDING INTERIOR INSPECTION",
    # "INSPECTION",
]

### Filter to HVAC rows only

In [ ]:
cond_cm = wr_fy["problem_type"].isin(CM_list)
cond_pm = wr_fy["problem_type"].isin(PM_list)

wr_HVAC = wr_fy[cond_cm | cond_pm]
wr_HVAC["is_pm"] = wr_HVAC["problem_type"].isin(PM_list)

print(f"We've gone from {len(wr_fy):,} rows to {len(wr_HVAC):,} rows.")

### Compute all PM/CM stats by fiscal year

In [ ]:
def compute_pm_cm(df, PM_list):
    df = df.copy().sort_values("fiscal_year")
    results_df = pd.DataFrame(
        columns=[
            "year",
            # "percent_pm",
            "pm_cm_ratio",
            "count_cm",
            "count_pm",
            "count_hvac",
        ]
    )
    for year in df["fiscal_year"].unique():
        results_dict = {}
        df_fy = df[df["fiscal_year"] == year]
        cond_pm = df_fy["is_pm"] == True
        count_pm = len(df_fy[cond_pm])
        count_hvac = len(df_fy)
        count_cm = count_hvac - count_pm
        results_dict["year"] = year
        # results_dict["percent_pm"] = (count_pm / count_hvac) * 100
        results_dict["pm_cm_ratio"] = count_pm / count_cm
        results_dict["count_pm"] = count_pm
        results_dict["count_cm"] = count_cm
        results_dict["count_hvac"] = count_hvac
        results_df = results_df.append(results_dict, ignore_index=True)
    results_df[["year", "count_cm", "count_pm", "count_hvac"]] = results_df[
        ["year", "count_cm", "count_pm", "count_hvac"]
    ].astype(int)
    return results_df.round(2)


pm_cm_results = compute_pm_cm(wr_HVAC, PM_list)

In [ ]:
cond_complete_FY = pm_cm_results["year"] <= 2020
pm_cm_results = pm_cm_results[cond_complete_FY]

pm_cm_results

In [ ]:
count_plot_data = pd.melt(
    pm_cm_results, id_vars=["year"], value_vars=["count_cm", "count_pm"]
)

sns.lineplot(data=count_plot_data, y="value", x="year", hue="variable")

sns.despine()

In [ ]:
ax = sns.lineplot(data=pm_cm_results, y="pm_cm_ratio", x="year",)
ax.set(title="Percent PM By Fiscal Year")
sns.despine()

## KPI 3: % of Work Orders Completed On-Time (CM)
Here are the key facts needed to understand the agency's traditional method for computing this KPI:

- Only CM problem types are considered, so all PM work orders are dropped.
- The on-time benchmark is 24 days

In [ ]:
wr_cm = wr_fy[~cond_pm]

print(f"We've gone from {len(wr_fy):,} rows to {len(wr_cm):,} rows.")

In [ ]:
wr_cm_on_time = compute_is_on_time(wr_cm, 24)

In [ ]:
pm_compliance = compute_kpi_table(wr_cm_on_time)
pm_compliance